In [1]:
import numpy as np

In [3]:
def convert_anc_count2(phgeno, anc):
    """
    Convert from ancestry and phased genotype to number of minor alles for each ancestry

    Args
    ----
    phgeno: n_indiv x 2n_snp, the first half columns contain the first haplotype,
        the second half columns contain the second haplotype
    anc: n_indiv x 2n_snp, match `phgeno`

    Returns
    ----
    geno: n_indiv x 2n_snp, the first half columns stores the number of minor alleles 
        from the first ancestry, the second half columns stores the number of minor
        alleles from the second ancestry
    """
    n_indiv = anc.shape[0]
    n_snp = anc.shape[1] // 2
    n_anc = 2
    geno = np.zeros_like(phgeno)
    for haplo_i in range(2):
        haplo_slice = slice(haplo_i * n_snp, (haplo_i + 1) * n_snp)
        haplo_phgeno = phgeno[:, haplo_slice]
        haplo_anc = anc[:, haplo_slice]
        for anc_i in range(n_anc):
            geno[:, (anc_i * n_snp) : ((anc_i + 1) * n_snp)][haplo_anc == anc_i] += haplo_phgeno[haplo_anc == anc_i]

    return geno

In [4]:
from os.path import join

In [5]:
DATA_DIR = "/u/project/pasaniuc/pasaniucdata/admixture/kangcheng/phenotype_simulation/out"
DATASET = "ukb_array/EUR_0.5_AFR_0.5_10_10000"
anc = np.load(join(DATA_DIR, DATASET, "anc.npy"))
phgeno = np.load(join(DATA_DIR, DATASET, "phgeno.npy"))

subset_index = np.arange(0, anc.shape[1], 1000)
anc = anc[:, subset_index]
phgeno = phgeno[:, subset_index]
n_indiv = anc.shape[0] // 2
n_snp = anc.shape[1]
anc = anc.reshape((n_indiv, n_snp * 2))
phgeno = phgeno.reshape((n_indiv, n_snp * 2))

In [38]:
import admix
anc = np.random.randint(0, 2, size=(10, 6))
phgeno = np.random.randint(0, 2, size=(10, 6))
count1 = admix.convert_anc_count(phgeno=phgeno, anc=anc)
count2 = convert_anc_count2(phgeno = phgeno, anc=anc)
assert np.all(count1 == count2)

In [32]:
n_sim = 10
beta, phe_g, phe = admix.simulate_admix_phenotype_cc(phgeno, anc, var_g=0., 
                                                     n_causal=1, cov=1.0, n_sim=n_sim, case_prevalence=0.2)

all_score_df = []
for sim_i in range(n_sim):

    mix_pheno = phe[:, sim_i]
    mix_anc = admix.add_up_haplotype(anc)
    mix_geno = admix.add_up_haplotype(phgeno)
    mix_theta = anc.mean(axis=1)
    
    # sample case control
    study_index = admix.sample_case_control(mix_pheno)
    score_df = admix.mixscore_wrapper(mix_pheno[study_index], mix_anc[study_index, :], mix_geno[study_index, :], 
                                      mix_theta[study_index])
    score_df["SIM_I"] = sim_i
    all_score_df.append(score_df)
all_score_df = pd.concat(all_score_df)

AttributeError: module 'admix' has no attribute 'simulate_admix_phenotype_cc'

In [ ]:
melted = all_score_df.iloc[:, 0:5].melt(var_name="score", value_name="val")
ax = sns.violinplot(x="score", y="val", data=melted)
means = melted.groupby(['score'])['val'].mean()